In [75]:
final_threshold = "<table class='paleGrayRows'> <thead> <tr> <th>Final Letter Grade</th> <th>Points Needed</th> <th>WebAssign Homework (entire course)</th> </tr> </thead> <tbody> <tr> <td>A</td> <td>≥ 51 points and no scores below G</td> <td>≥ 80%</td> </tr> <tr> <td>B</td> <td>≥ 45 points and at most 2 scores below G</td> <td>≥ 70%</td> </tr> <tr> <td>C</td> <td>≥ 39 points and at most 6 scores below G</td> <td>≥ 60%</td> </tr> <tr> <td>D</td> <td>≥ 33 points</td> <td>≥ 50%</td> </tr> <tr> <td>F</td> <td>&lt; 33 points</td> <td>&lt; 50%</td> </tr> </tbody> </table>"
midterm_threshold = "<table class='paleGrayRows'> <thead> <tr> <th>Midterm Letter Grade</th> <th>Course Standards</th> <th>WebAssign Homework (12.1-12.4 Chapter 6, Chapter 7, 8.1, 8.2)</th> </tr> </thead> <tbody> <tr> <td>A</td> <td>≥ 19 points and no scores below G</td> <td>≥ 80%</td> </tr> <tr> <td>B</td> <td>≥ 17 points and at most 2 scores below G</td> <td>≥ 70%</td> </tr> <tr> <td>C</td> <td>≥ 13 points and at most 4 scores below G</td> <td>≥ 60%</td> </tr> <tr> <td>D</td> <td>≥ 11 points</td> <td>≥ 50%</td> </tr> <tr> <td>F</td> <td>&lt; 11 points</td> <td>&lt; 50%</td> </tr> </tbody> </table>"

In [1]:
import pandas as pd


df = pd.read_csv("test.csv")
better_cols = ['Last Name', 'First Name', 'Username', 'Student ID', 'Last Access', 'Availability', '01', 'V1', 'V2', 'V3', 'V4', 'F1', 'F2', 'F3', 'F4', 'I1', 'I2', 'I3', 'I4', 'I5', 'I6', 'A1', 'A2', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'P1', 'P2', 'P3', 'C1', 'C2', 'C3', 'C4', 'C5']
df.columns = better_cols

## Make a better index column and dump everything not grade-related
df['Name'] = df['Last Name'] + ', ' + df['First Name']
df.drop(['Last Name', 'First Name', 'Username', 'Student ID', 'Last Access', 'Availability'], axis=1, inplace=True)
df.set_index("Name", inplace=True)


In [2]:
## Define the tools for reading the strings into numeric values: 

def sgn_to_number(sgn):
#Note that S has the value 1, G has the value 0.5, and N has the value 0; else is 0; 
    if (sgn == "S"):
        return 1
    elif (sgn == "G"):
        return 0.5
    else:
        return 0
    
def marks_to_score(marks):
    ## First, extract the last two entries:
    try: 
        marks = marks[-2:]
    except TypeError:  # encountered NaN or not a string
        return 0
    except IndexError: # encountered NaN; bah
        return 0
    
    return {"score":sum(map(sgn_to_number, marks)), "number of ns": marks.count("N")+marks.count("Z")}
    
    
## Make sure nothing broke 
assert(marks_to_score("SSNSGS") == {"score":1.5, "number of ns":0})

In [3]:
def summative(marks):
    score_so_far = 0
    ns_so_far = 0
    for cell in marks:
        try: 
            score_so_far += cell['score']
            ns_so_far += cell['number of ns']
        except:
            pass
    return {'Score':score_so_far, 'Number of ns': ns_so_far}

In [70]:
class_scores = list()
for index, row in df.iterrows():
    raw = pd.DataFrame({'Standard':row.index, 'Marks':row.values})
    raw = raw.dropna()
    class_scores.append( {"midterm_or_final":"Midterm", "Name":index, "Marks":summative(row.apply(marks_to_score)), "Raw Data":raw})

## Dataset for this is a hashset
## Key      |   Data
## ---------+---------
## Name     |  full name:  last, first
## Marks    |  hash of {'Score' and 'Number of ns'}
## Raw Data |  full pandas Series whose index is the Name above

In [95]:
# Now let's export the PDF report:
from jinja2 import Environment, FileSystemLoader
from weasyprint import HTML
from datetime import date


## grade threshold temporary stuff:
grade_threshold = midterm_threshold


def print_the_report_for(individual):
    
    # update the date in the off chance it runs between cusp of one day and the next
    today = date.today().strftime("%B %d, %Y")
    env = Environment(loader=FileSystemLoader('.'))
    template = env.get_template("template.html")
    template_vars = {
        "Name": individual['Name'],
        "raw_data": individual['Raw Data'].to_html(index=False, bold_rows=False, classes=["paleGrayRows"]),
        "score": individual['Marks']['Score'],
        "number_of_ns": individual['Marks']['Number of ns'],
        "letter_grade": "QQQQQQQQQQ",
        "date": today,
        "midterm_or_final": individual['midterm_or_final'],
        "grade_threshold": final_threshold
    }

    output = template.render(template_vars)
    #print(output)
    HTML(string=output).write_pdf("test.pdf",stylesheets=["./report.css"])
    
print_the_report_for(class_scores[13])